In [1]:
from STK_Sim import *

Filename = 'AERO_401_Initial_Assessment'

stk_object = STK_Simulation(True,Filename)

start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
dt = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+dt).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time

Loading STK...
----------------------------------------------------------------------------------------------------
Loaded STK
----------------------------------------------------------------------------------------------------
Loading Scenario...
----------------------------------------------------------------------------------------------------
Loaded Scenario
----------------------------------------------------------------------------------------------------


In [2]:
Target_Load_Option=0
Num_Targets=15
lon_range=(0,90)
lat_range=(0,90) 
stk_object.Target_Loader(Target_Load_Option,Num_Targets,lon_range,lat_range)
print(stk_object.Target_Initial_Data)

Loading Targets...
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Loaded Targets
           LAT      LON
TAR##                  
1      10.8082  33.8156
2       2.1675  43.3405
3      41.8441  64.1956
4      81.8933  63.1415
5      17.4575  15.1535
6      89.4669  50.2139
7      50.7925  22.4388
8      82.9137  51.3838
9      62.0530  12.2774
10     58.7236   1.8884
11     80.5024   9.3503
12     37.5831  33.5028
13     31.1283   0.1648
14     64.9297  22.9369
15     43.8052   5.4436


In [3]:
Satellite_Load_Option=1
Num_Satellites=12
Orbit = 'Circular'
aop_range=(0,0)    #Shifts the satellite left and right in its same path
asc_range=(0,90)    #Shifts the path left and right
loc_range=(0,0)      #Shifts the satellite left and right in its same path
targeting = True
if Orbit == 'Circular':
    rev_range=(10,15)
    stk_object.Circular_Satellite_Loader(Satellite_Load_Option,Num_Satellites,rev_range,aop_range,asc_range,loc_range,targeting)
else:
    per_range=(600,600)
    apo_range=(2000,2000)
    inc_range=(75,75)
    stk_object.Elliptical_Satellite_Loader(Satellite_Load_Option,Num_Satellites,per_range,apo_range,inc_range,aop_range,asc_range,loc_range,targeting)
print(stk_object.Satellite_Initial_Data)

Loading Satellites...
----------------------------------------------------------------------------------------------------
Loaded Satellites
----------------------------------------------------------------------------------------------------
             REV          ALT         INC  AOP        ASC  LOC   TAR
SAT##                                                               
1      10.000000  2722.560987  110.074262  0.0   0.000000  0.0  True
2      10.454545  2456.827228  108.024339  0.0   8.181818  0.0  True
3      10.909091  2209.678762  106.270358  0.0  16.363636  0.0  True
4      11.363636  1979.119505  104.756704  0.0  24.545455  0.0  True
5      11.818182  1763.436745  103.440747  0.0  32.727273  0.0  True
6      12.272727  1561.151854  102.289200  0.0  40.909091  0.0  True
7      12.727273  1370.981024  101.275666  0.0  49.090909  0.0  True
8      13.181818  1191.803738  100.378942  0.0  57.272727  0.0  True
9      13.636364  1022.637213   99.581804  0.0  65.454545  0.0  True

In [4]:
Cd=2.2
Cr=1.0
DragArea=13.65
SunArea=15.43
Mass=1000.0
stk_object.Compute_Lifetime(Cd,Cr,DragArea,SunArea,Mass)
print(stk_object.Lifetimes)

Computing Lifetimes...
----------------------------------------------------------------------------------------------------
Computed Lifetimes
----------------------------------------------------------------------------------------------------
        Orbits        Time
SAT##                     
1       >99999     -------
2       >99999     -------
3       >99999     -------
4       >99999     -------
5       >99999     -------
6       >99999     -------
7       >99999     -------
8       >99999     -------
9       >99999     -------
0       >99999     -------
1       >99999     -------
2      66376.0  12.0 years


Plotting Ground Track

In [5]:
# dt = 60
# stk_object.Plot(dt)

In [6]:
dt = 60
stk_object.Compute_AzEl(dt)

Computing AzEl
----------------------------------------------------------------------------------------------------
Computed Access
----------------------------------------------------------------------------------------------------


In [9]:
fig=go.Figure()
fig.update_layout(showlegend=False,scene=dict(
                                                xaxis_title='Target Access',
                                                yaxis_title='Azimuth (Increments of 10°)',
                                                zaxis_title='Count',
                                            ),)

for j in stk_object.targets:
    x=list(stk_object.targets.keys()).index(j)
    y_list = []
    z_list = []
    for i in stk_object.satellites:
        if f'{j}->{i}' in list(stk_object.AzEl_data.keys()):
            df = stk_object.AzEl_data[f'{j}->{i}']
            df = pd.DataFrame(df)
            y=np.linspace(0,90,10)
            z=np.histogram(df['Azimuth'].astype(float), bins=y, density=False)[0].tolist()
            z=np.repeat(z,2).tolist()
            z.insert(0,0)
            z.pop()
            y_list.extend(y)
            z_list.extend(z)

    fig.add_traces(go.Scatter3d(x=[x]*len(z), y=np.repeat(y,2), z=z,mode='lines',name=j))
fig.update_traces(line={'width': 30})
fig.show(renderer='browser')

In [10]:
fig=go.Figure()
fig.update_layout(showlegend=False,scene=dict(
                                                xaxis_title='Target Access',
                                                yaxis_title='Elevation (Increments of 10°)',
                                                zaxis_title='Count',
                                            ),)

for j in stk_object.targets:
    x=list(stk_object.targets.keys()).index(j)
    y_list = []
    z_list = []
    for i in stk_object.satellites:
        if f'{j}->{i}' in list(stk_object.AzEl_data.keys()):
            df = stk_object.AzEl_data[f'{j}->{i}']
            df = pd.DataFrame(df)
            y=np.linspace(0,180,10)
            z=np.histogram(df['Elevation'].astype(float), bins=y, density=False)[0].tolist()
            z=np.repeat(z,2).tolist()
            z.insert(0,0)
            z.pop()
            y_list.extend(y)
            z_list.extend(z)

    fig.add_traces(go.Scatter3d(x=[x]*len(z), y=np.repeat(y,2), z=z,mode='lines',name=j))
fig.update_traces(line={'width': 30})
fig.show(renderer='browser')